This is just a quick notebook you can read, there's no tasks, but hopefully it might get you thinking about some useful things that can be done with GPUs for galaxy spectral analysis.

One of the most interesting things about Colab is that it gives you free access to a GPU, as well as being pre-installed with Google's Tensorflow Python package for machine learning and GPU-accelerated computing.

I've put together here a very quick example of how this might be useful for spectral fitting. This is just focused on making linear algebra happen more quickly on a GPU, rather than by applying machine learning methods, which is probably the most common application for Tensorflow.

Before running anything in this notebook, go to Edit -> Notebook settings and check Hardware accelerator is set to GPU.

On testing this notebook, I've found the results vary quite a lot when re-running, or after factory reseting the runtime. I guess this is partially because different hardware gets assigned to the notebook at different times. It might be worth using the options under Runtime to test a few different runtimes and repeated runs of the cells.



In [ ]:
# This first cell sets up the virtual machine with all the necessary software.
# There should be no need to edit this cell, just scroll down...

# Install Bagpipes and python dependencies
!pip install bagpipes

# Adjust the output height to avoid a huge wall of installation text
from IPython import display
display.Javascript("google.colab.output.setIframeHeight('100px');")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.0/206.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 7.7 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

In [ ]:
# Ignore the MultiNest errors

import numpy as np
import bagpipes as pipes
import time
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

gpu = tf.config.list_logical_devices(device_type="GPU")[0]
cpu = tf.config.list_logical_devices(device_type="CPU")[0]

BAGPIPES: Generating IGM absorption table.
Bagpipes: Latex distribution not found, plots may look strange.
Bagpipes: PyMultiNest import failed, fitting will use the Nautilus sampler instead.


The code below generates some random star-formation and chemical enrichment histories, then collapses the BC03 stellar grid into a 1D spectrum with them using a tensorflow function.

The process is repeated on a CPU and a GPU. Play around with the number of models and number of wavelength points. How does the time taken per model scale with these two quantities on the CPU and GPU? What regime might it be worth using the GPU in? Remember, it's usually fairly easy to get hold of multiple CPUs at once, whereas getting access to lots of GPUs at once might be a bit more difficult.

In [ ]:
# Set how many models to generate and how many wavelength points per spectrum
n_models = 1000
n_wav_points = 1000

print("\nNumber of wavelength grid points:", n_wav_points)
print("Making", n_models, "models\n")

# Make wavelength sampling
wavs = np.logspace(1, 8, n_wav_points)

# Initialise bagpipes stellar model to get access to the BC03 stellar grid
stellar_model = pipes.models.stellar(wavs)


@tf.function(experimental_compile=True)
def spectrum_tf(sfh_ceh, grid):
    """ tensorflow function to collapse stellar grid and SFH into a spectrum """
    spectrum = tf.math.reduce_sum(tf.math.reduce_sum(grid*sfh_ceh, axis=2), axis=1)

    return spectrum


# Make some random star-formation and chemical enrichment histories
sfh_grid = tf.constant(np.random.randn(n_models, 7, 42) + 10.)

# Pull the BC03 stellar model grid out of bagpipes
stellar_grid = tf.constant(stellar_model.grid)
print("Stellar grid dimensions (wavelength, metallicity, age)", stellar_grid.shape, "\n")

# Time grid making on cpu
time0 = time.time()

with tf.device(cpu):
    for i in range(n_models):
        spectrum_tf(sfh_grid[i, :, :], stellar_grid)

time_stellar_tf_cpu = (time.time() - time0)/n_models
print("Average time cpu:", "%.6f" % time_stellar_tf_cpu, "seconds.")


# Time grid making on gpu
time0 = time.time()

with tf.device(gpu):
    for i in range(n_models):
        spectrum_tf(sfh_grid[i, :, :], stellar_grid)

time_stellar_tf_gpu = (time.time() - time0)/n_models
print("Average time gpu:", "%.6f" % time_stellar_tf_gpu, "seconds.")
print("gpu speed =", "%.3f" % (time_stellar_tf_cpu/time_stellar_tf_gpu), "x cpu speed")



Number of wavelength grid points: 1000
Making 1000 models

Stellar grid dimensions (wavelength, metallicity, age) (1000, 7, 42) 

Average time cpu: 0.001921 seconds.
Average time gpu: 0.001691 seconds.
gpu speed = 1.136 x cpu speed


Of course, in the above example, each model is generated sequentially, as is necessary if you're using a sampling routine that tells you which models it wants generated. What about if we can generate all n_models at once, as we would, for example, with a grid-based code.

In [ ]:
# Set how many models to generate and how many wavelength points per spectrum
n_models = 1000
n_wav_points = 1000

print("\nNumber of wavelength grid points:", n_wav_points)
print("Making", n_models, "models\n")

# Make wavelength sampling
wavs = np.logspace(1, 8, n_wav_points)

# Initialise bagpipes stellar model to get access to the BC03 stellar grid
stellar_model = pipes.models.stellar(wavs)


@tf.function(experimental_compile=True)
def spectrum_grid_tf(sfh_ceh, grid):
    """ Collapse stellar grid and N x SFH into N x spectra """
    spectrum_grid = tf.math.reduce_sum(tf.math.reduce_sum(grid*sfh_ceh, axis=2), axis=1)

    return spectrum_grid


# Make some random star-formation and chemical enrichment histories
sfh_grid = tf.constant(np.random.randn(1, 7, 42, n_models) + 10.)

# Pull the BC03 stellar model grid out of bagpipes
stellar_grid = np.expand_dims(tf.constant(stellar_model.grid), axis=-1)

# Time grid making on cpu
time0 = time.time()

with tf.device(cpu):
    spectrum_grid = spectrum_grid_tf(sfh_grid, stellar_grid)

time_stellar_tf_cpu = (time.time() - time0)/n_models

print("Average time cpu:", "%.6f" % time_stellar_tf_cpu, "seconds.")


# Time grid making on gpu
time0 = time.time()

with tf.device(gpu):
    spectrum_grid = spectrum_grid_tf(sfh_grid, stellar_grid)

time_stellar_tf_gpu = (time.time() - time0)/n_models

print("Average time gpu:", "%.6f" % time_stellar_tf_gpu, "seconds.")
print("gpu speed =", "%.3f" % (time_stellar_tf_cpu/time_stellar_tf_gpu), "x cpu speed")



Number of wavelength grid points: 1000
Making 1000 models

Average time cpu: 0.002117 seconds.
Average time gpu: 0.000173 seconds.
gpu speed = 12.272 x cpu speed
